## Notebook for downloading and importing AusTraits data into specieslist lists

__Aim of script__: to extract a list of taxa with trait data for a single "trait" and upload the compiled CSV file to an existing species list (replacing rows with updated data).

AusTraits Swagger docs: [`http://traitdata.austraits.cloud.edu.au/__docs__/#`](http://http://traitdata.austraits.cloud.edu.au/__docs__/#)

`download_trait_data` service URL: `http://traitdata.austraits.cloud.edu.au/download-trait-data`

Example POST: 
`curl -X POST "http://traitdata.austraits.cloud.edu.au/download-trait-data" -H "accept: */*" -H "Content-Type: application/json" -d "{\"taxa\":[],\"traits\":[\"post_fire_recruitment\"]}"`

POST body example:
```json
{
  "traits": [
    "post_fire_recruitment"
  ]
}
```

AusTraits trait definitions: https://traitecoevo.github.io/austraits.build/articles/trait_definitions.html#fire_response

ALA list tool Swagger docs: https://docs.ala.org.au/openapi/index.html?urls.primaryName=specieslist

POST to https://docs.ala.org.au/openapi/index.html?urls.primaryName=specieslist#/Lists/Add%20or%20replace%20a%20species%20list

**NOTE**: posting data to lists.ala.org.au requires a `.env` file at the repository root, that will contain sensitive data.
Use the `.env.template` as a starting point and do NOT add the `.env` to git (`.gitignore` will prevent this).

### TODO fixes and improvements

- [ ] Convert to vanilla Python script for easier automating and running via Airflow, etc.
- [ ] Use command-line arg for `trait_name` variable and type-check input via Enum 
- [x] Fix duplicate values after processing: `fire_killed|fire_killed`, `resprouts|resprouts`, `resprouts|resprouts|resprouts`
- [x] Fix values with same "items" with different order - combine to single "set", e,g, `fire_killed|resprouts` and `resprouts|fire_killed`. Put in `Set` first?
- [x] Fix remove values of `unknown` (convert to empty string)
- [-] Move all URLs into config - so `test` vs `prod` URLs can be specified
- [-] Move references to list DRs IDs into config
- [x] Move Oauth/JWT code into helper script in `code` directory
- [x] Add code to regenerate the `access_token` using the `refresh_token` 

In [65]:
import requests
import pandas as pd
import io
from enum import Enum
import decouple

config = decouple.AutoConfig(' ') # this is a hack for Jupyter due to running in a sandbox

# Lists-test DRs 
# TODO: move in ENVs similar to `ACCESS_TOKEN`?

Traits = Enum("Traits",
    [
        ("fire_response", config('DR_FIRE_RESPONSE', default="dr18689")),
        ("post_fire_recruitment", config('DR_POST_FIRE_RECRUITMENT', default="dr18693")),
        ("photosynthetic_pathway", config('DR_PHOTOSYNTHETIC_PATHWAY', default="dr18707")),
    ]
)

# this_trait = Traits['post_fire_recruitment'] # TODO: handle via args
this_trait = Traits['fire_response'] # TODO: handle via args
trait_name = this_trait.name # "fire_response" | "post_fire_recruitment"
trait_dr_id = this_trait.value

austraits_base_url = config('AUSTRAITS_BASE_URL', default="http://traitdata.austraits.cloud.edu.au")
url = f"{austraits_base_url}/download-trait-data"
json_body = { "traits": [ trait_name ] }
print("url", url, json_body)
exit
response = requests.post(url, json = json_body)
df = pd.read_csv(io.StringIO(response.text), sep=",", dtype=str, keep_default_na=False)
total_rows = len(df.index)
print("Total rows =",total_rows)
df.info()

url http://traitdata.austraits.cloud.edu.au/download-trait-data {'traits': ['fire_response']}
Total rows = 19500
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19500 entries, 0 to 19499
Data columns (total 53 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   dataset_id                        19500 non-null  object
 1   taxon_name                        19500 non-null  object
 2   observation_id                    19500 non-null  object
 3   trait_name                        19500 non-null  object
 4   trait_value                       19500 non-null  object
 5   unit                              19500 non-null  object
 6   entity_type                       19500 non-null  object
 7   value_type                        19500 non-null  object
 8   basis_of_value                    19500 non-null  object
 9   replicates                        19500 non-null  object
 10  basis_of_record              

Sort the table on taxon_name, so we can see the duplicate entries on `taxon_name`

In [66]:
df.sort_values(by=['taxon_name'], ascending=True, inplace=True)
df.head(20)

,dataset_id,taxon_name,observation_id,trait_name,trait_value,unit,entity_type,value_type,basis_of_value,replicates,...,binomial,genus,family,taxon_distribution,establishment_means,taxonomic_status,scientific_name,scientific_name_authorship,taxon_id,scientific_name_id
1667,Clarke_2015,Abelmoschus ficulneus,0001,fire_response,fire_killed,,population,mode,expert_score,,...,Abelmoschus ficulneus,Abelmoschus,Malvaceae,"WA, NT, Qld",native,accepted,Abelmoschus ficulneus (L.) Wight,(L.) Wight,https://id.biodiversity.org.au/node/apni/2897916,https://id.biodiversity.org.au/name/apni/55929
1668,Clarke_2015,Abelmoschus moschatus,0002,fire_response,resprouts,,population,mode,expert_score,,...,Abelmoschus moschatus,Abelmoschus,Malvaceae,"WA, NT, Qld, NSW (naturalised)",native and naturalised,accepted,Abelmoschus moschatus Medik.,Medik.,https://id.biodiversity.org.au/node/apni/2900572,https://id.biodiversity.org.au/name/apni/55953
13254,White_2020,Abrodictyum caudatum,1747,fire_response,fire_killed,,species,mode,expert_score,,...,Abrodictyum caudatum,Abrodictyum,Hymenophyllaceae,"Qld, NSW, Vic",native,accepted,Abrodictyum caudatum (Brack.) Ebihara & K.Iwats.,(Brack.) Ebihara & K.Iwats.,https://id.biodiversity.org.au/node/apni/7402200,https://id.biodiversity.org.au/name/apni/241954
16314,White_2020,Abrodictyum caudatum,4818,fire_response,fire_killed,,species,mode,expert_score,,...,Abrodictyum caudatum,Abrodictyum,Hymenophyllaceae,"Qld, NSW, Vic",native,accepted,Abrodictyum caudatum (Brack.) Ebihara & K.Iwats.,(Brack.) Ebihara & K.Iwats.,https://id.biodiversity.org.au/node/apni/7402200,https://id.biodiversity.org.au/name/apni/241954
11515,White_2020,Abrotanella,0005,fire_response,resprouts,,species,mode,expert_score,,...,,Abrotanella,Asteraceae,,,accepted,Abrotanella Cass.,Cass.,https://id.biodiversity.org.au/taxon/apni/5126...,https://id.biodiversity.org.au/name/apni/56103
5019,Kirkpatrick_2020,Abrotanella forsteroides,001,fire_response,resprouts,,species,mode,expert_score,,...,Abrotanella forsteroides,Abrotanella,Asteraceae,Tas,native,accepted,Abrotanella forsteroides (Hook.f.) Benth.,(Hook.f.) Benth.,https://id.biodiversity.org.au/node/apni/2901344,https://id.biodiversity.org.au/name/apni/56111
11514,White_2020,Abrotanella nivigena,0004,fire_response,resprouts,,species,mode,expert_score,,...,Abrotanella nivigena,Abrotanella,Asteraceae,"NSW, Vic",native,accepted,Abrotanella nivigena (F.Muell.) F.Muell.,(F.Muell.) F.Muell.,https://id.biodiversity.org.au/node/apni/2900512,https://id.biodiversity.org.au/name/apni/56120
10752,Schmidt_2003,Abrus precatorius,0005,fire_response,resprouts,,species,mode,expert_score,,...,Abrus precatorius,Abrus,Fabaceae,"WA, NT, Qld, NSW",native,accepted,Abrus precatorius L.,L.,https://id.biodiversity.org.au/node/apni/2919311,https://id.biodiversity.org.au/name/apni/56149
11530,White_2020,Abutilon,0020,fire_response,fire_killed,,species,mode,expert_score,,...,,Abutilon,Malvaceae,,,accepted,Abutilon Mill.,Mill.,https://id.biodiversity.org.au/taxon/apni/5143...,https://id.biodiversity.org.au/name/apni/56205
11516,White_2020,Abutilon calliphyllum,0006,fire_response,fire_killed,,species,mode,expert_score,,...,Abutilon calliphyllum,Abutilon,Malvaceae,Qld,native,accepted,Abutilon calliphyllum Domin,Domin,https://id.biodiversity.org.au/node/apni/3835441,https://id.biodiversity.org.au/name/apni/56304


In [67]:
print(df[df['taxon_name'].str.contains('Panicum')])

         dataset_id                                  taxon_name  \
16891    White_2020                                     Panicum   
16875    White_2020                              Panicum buncei   
16879    White_2020                        Panicum decompositum   
9760    NSWFRD_2014                        Panicum decompositum   
16876    White_2020      Panicum decompositum var. decompositum   
16877    White_2020           Panicum decompositum var. tenuius   
16878    White_2020           Panicum decompositum var. tenuius   
1014     Cheal_2017                             Panicum effusum   
6094   Moore_2019_2                             Panicum effusum   
6085   Moore_2019_2                             Panicum effusum   
6086   Moore_2019_2                             Panicum effusum   
6087   Moore_2019_2                             Panicum effusum   
6088   Moore_2019_2                             Panicum effusum   
6089   Moore_2019_2                             Panicum effusu

### Clean-up data
 - Remove rows with `trait_value` = "unknown" - treat as if no entry existed
 - Remove rows with `taxon_name` containing `sp.` desingations - we don't want higher taxon matches for now.
 - See a list of distinct values for the trait value column

In [55]:
print("before:", len(df.index))
df = df[df.trait_value != "unknown"]
df = df[~df['taxon_name'].str.contains(r'\s+sp\.')]
print("after:", len(df.index))
print("unique traits:", df['trait_value'].unique())

before: 13632
after: 13243
unique traits: ['c3' 'c4' 'cam' 'c3-c4' 'c4-cam' 'c3-cam' 'facultative_cam' 'c3 cam'
 'c3 c4']


Extract only the few columns of interest and rename column names for importing into lists app.

In [56]:
# df2 = df[['taxon_name','binomial','taxon_rank','trait_name','trait_value','location']].copy()
df2 = df.rename(columns={'taxon_name':'taxon','trait_name': 'traitName', 'trait_value': 'traitValue'})

df2.head(80)

,dataset_id,taxon,observation_id,traitName,traitValue,unit,entity_type,value_type,basis_of_value,replicates,...,binomial,genus,family,taxon_distribution,establishment_means,taxonomic_status,scientific_name,scientific_name_authorship,taxon_id,scientific_name_id
9826,White_2020,Abrodictyum caudatum,4818,photosynthetic_pathway,c3,,species,mode,expert_score,,...,Abrodictyum caudatum,Abrodictyum,Hymenophyllaceae,"Qld, NSW, Vic",native,accepted,Abrodictyum caudatum (Brack.) Ebihara & K.Iwats.,(Brack.) Ebihara & K.Iwats.,https://id.biodiversity.org.au/node/apni/7402200,https://id.biodiversity.org.au/name/apni/241954
6886,White_2020,Abrodictyum caudatum,1747,photosynthetic_pathway,c3,,species,mode,expert_score,,...,Abrodictyum caudatum,Abrodictyum,Hymenophyllaceae,"Qld, NSW, Vic",native,accepted,Abrodictyum caudatum (Brack.) Ebihara & K.Iwats.,(Brack.) Ebihara & K.Iwats.,https://id.biodiversity.org.au/node/apni/7402200,https://id.biodiversity.org.au/name/apni/241954
5192,White_2020,Abrotanella nivigena,0004,photosynthetic_pathway,c3,,species,mode,expert_score,,...,Abrotanella nivigena,Abrotanella,Asteraceae,"NSW, Vic",native,accepted,Abrotanella nivigena (F.Muell.) F.Muell.,(F.Muell.) F.Muell.,https://id.biodiversity.org.au/node/apni/2900512,https://id.biodiversity.org.au/name/apni/56120
4911,Schmidt_2003,Abrus precatorius,0005,photosynthetic_pathway,c3,,species,mode,expert_score,,...,Abrus precatorius,Abrus,Fabaceae,"WA, NT, Qld, NSW",native,accepted,Abrus precatorius L.,L.,https://id.biodiversity.org.au/node/apni/2919311,https://id.biodiversity.org.au/name/apni/56149
5207,White_2020,Abutilon,0020,photosynthetic_pathway,c3,,species,mode,expert_score,,...,,Abutilon,Malvaceae,,,accepted,Abutilon Mill.,Mill.,https://id.biodiversity.org.au/taxon/apni/5143...,https://id.biodiversity.org.au/name/apni/56205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5234,White_2020,Acacia bivenosa,0047,photosynthetic_pathway,c3,,species,mode,expert_score,,...,Acacia bivenosa,Acacia,Fabaceae,"WA, NT, Qld",native,accepted,Acacia bivenosa DC.,DC.,https://id.biodiversity.org.au/node/apni/2912987,https://id.biodiversity.org.au/name/apni/59491
5233,White_2020,Acacia bivenosa subsp. wayi,0046,photosynthetic_pathway,c3,,species,mode,expert_score,,...,,Acacia,Fabaceae,,,accepted,Acacia Mill.,Mill.,https://id.biodiversity.org.au/taxon/apni/5147...,https://id.biodiversity.org.au/name/apni/56859
5236,White_2020,Acacia boormanii,0049,photosynthetic_pathway,c3,,species,mode,expert_score,,...,Acacia boormanii,Acacia,Fabaceae,"NSW, ACT (naturalised), Vic",native and naturalised,accepted,Acacia boormanii Maiden,Maiden,https://id.biodiversity.org.au/taxon/apni/5143...,https://id.biodiversity.org.au/name/apni/59572
1,Cunningham_1999,Acacia brachybotrya,006,photosynthetic_pathway,c3,,species,mode,expert_score,,...,Acacia brachybotrya,Acacia,Fabaceae,"SA, NSW, Vic",native,accepted,Acacia brachybotrya Benth.,Benth.,https://id.biodiversity.org.au/node/apni/2892012,https://id.biodiversity.org.au/name/apni/59600


Transform data for taxa that have multiple entries with different values in `traitValue` column - combining all values into a pipe-delimited column.
See https://www.statology.org/pandas-combine-rows-with-same-column-value/

Debugging section - delete if no longer required

In [57]:
df_tmp = df2.loc[df2['traitValue'] == "fire_killed resprouts"]
print(df_tmp[['taxon','traitValue']].to_string(index=False)) 
# df_Solanum = df[df['taxon'] == 'Solanum']
# print(df_Solanum.head())
contain_values = df2[df2['taxon'].str.contains(r'\s+sp\.')]
print (len(contain_values))

Empty DataFrame
Columns: [taxon, traitValue]
Index: []
0


In [58]:
# cleanup data for `traitValue` with values that are already mulit-valued (shouldn't be, data should have 1-to-1 ratio of rows to traits
# #     col1  col2
# 0  a,b,c     1
# 1  d,e,f     2
# df.assign(col1 = df.col1.str.split(',')).explode('col1', ignore_index=True)
print("before", df2['traitValue'].unique(), len(df2.index))

df3 = df2.assign(traitValue = df2.traitValue.str.split()).explode('traitValue', ignore_index=True).reset_index(drop=True).ffill()
# df4 = df.assign(traitValue=df['traitValue'].str.split()).explode('traitValue')
print("after 1", df3['traitValue'].unique(), len(df3.index))
# print("after 2", df4['traitValue'].unique(), len(df4.index))
# df.head(5)

before ['c3' 'c4' 'cam' 'c3-c4' 'c4-cam' 'c3-cam' 'facultative_cam' 'c3 cam'
 'c3 c4'] 13243
after 1 ['c3' 'c4' 'cam' 'c3-c4' 'c4-cam' 'c3-cam' 'facultative_cam'] 13248


In [59]:

#df['traitValue'] = df['traitValue'].str.replace('fire_killed resprouts', 'fire_killed|resprouts')
#     col1  col2
# 0  a,b,c     1
# 1  d,e,f     2
# df.assign(col1 = df.col1.str.split(',')).explode('col1', ignore_index=True)
# df.assign(traitValue = df.traitValue.str.split(' ')).explode('traitValue', ignore_index=True)
# This groupBy combines rows with the same `taxon` value and adds the `traitValue` for each to a set, 
# then converts to a list and sorts it, and then joins with `|` to a string value. 
# This is to prevernt seeing values of `fire_killed|resprouts` and `resprouts|fire_killed` in the index (effectively the same)
df4 = df3.groupby(['taxon','traitName'])['traitValue'].agg(lambda x: sorted(list(set(x)))).apply('|'.join).reset_index()
# df2 = df.groupby(['taxon','traitName'])['traitValue'].agg(lambda x: set(x)).apply('|'.join).reset_index()
# df2['traitValue'] = df['traitValue'].str.replace('unknown', '')
grouped_rows = len(df4.index)
print("Total rows =",len(df.index))
print("Grouped rows =",grouped_rows)
print(df4['traitValue'].unique())
df4.head(50)
# df3.head(20)

Total rows = 13243
Grouped rows = 8547
['c3' 'c3|c4' 'c3|cam' 'c4' 'cam' 'c3|c3-c4' 'c4|c4-cam' 'c4|cam'
 'c3-cam|cam' 'c3-cam' 'cam|facultative_cam' 'c3|cam|facultative_cam'
 'c3-c4' 'c3-c4|cam']


,taxon,traitName,traitValue
0,Abrodictyum caudatum,photosynthetic_pathway,c3
1,Abrotanella nivigena,photosynthetic_pathway,c3
2,Abrus precatorius,photosynthetic_pathway,c3
3,Abutilon,photosynthetic_pathway,c3
4,Abutilon calliphyllum,photosynthetic_pathway,c3
5,Abutilon fraseri,photosynthetic_pathway,c3
6,Abutilon fraseri subsp. diplotrichum,photosynthetic_pathway,c3
7,Abutilon fraseri subsp. fraseri,photosynthetic_pathway,c3
8,Abutilon halophilum,photosynthetic_pathway,c3
9,Abutilon hannii,photosynthetic_pathway,c3


In [60]:
df4.to_csv(f"/data/arga-data/aus_traits_{trait_name}.csv", index=False)

In [61]:
json_records = df4.to_dict('tight')
# print("json_records:", list(json_records.items())[:10])
print("json_records:",json_records['data'][:40])

json_records: [['Abrodictyum caudatum', 'photosynthetic_pathway', 'c3'], ['Abrotanella nivigena', 'photosynthetic_pathway', 'c3'], ['Abrus precatorius', 'photosynthetic_pathway', 'c3'], ['Abutilon', 'photosynthetic_pathway', 'c3'], ['Abutilon calliphyllum', 'photosynthetic_pathway', 'c3'], ['Abutilon fraseri', 'photosynthetic_pathway', 'c3'], ['Abutilon fraseri subsp. diplotrichum', 'photosynthetic_pathway', 'c3'], ['Abutilon fraseri subsp. fraseri', 'photosynthetic_pathway', 'c3'], ['Abutilon halophilum', 'photosynthetic_pathway', 'c3'], ['Abutilon hannii', 'photosynthetic_pathway', 'c3'], ['Abutilon leucopetalum', 'photosynthetic_pathway', 'c3'], ['Abutilon malvifolium', 'photosynthetic_pathway', 'c3'], ['Abutilon otocarpum', 'photosynthetic_pathway', 'c3'], ['Abutilon oxycarpum', 'photosynthetic_pathway', 'c3'], ['Abutilon oxycarpum f. acutatum', 'photosynthetic_pathway', 'c3'], ['Abutilon oxycarpum var. incanum', 'photosynthetic_pathway', 'c3'], ['Abutilon oxycarpum var. oxycarpum'

Output from json_records:
```json
 [
  [
    "Abelmoschus ficulneus",
    "fire_response",
    "fire_killed"
  ],
  [
    "Abelmoschus moschatus",
    "fire_response",
    "resprouts"
  ],
  [
    "Abrodictyum caudatum",
    "fire_response",
    "fire_killed"
  ],
  [
    "Abrotanella",
    "fire_response",
    "resprouts"
  ]
]
```

Munge data into required format (see below)

```json
[
  {
    "itemName": "item1",
    "kvpValues": [
      {
        "key": "key1",
        "value": "value1"
      },
      {
        "key": "key2",
        "value": "value2"
      }
    ]
  },
  {
    "itemName": "item2",
    "kvpValues": [
      {
        "key": "key3",
        "value": "value3"
      },
      {
        "key": "key4",
        "value": "value4"
      }
    ]
  }
]
```

In [62]:
list_items = []

#for taxon_record in json_records['data']:
limit = 1000000 # lower for debugging
for count, taxon_record in enumerate(json_records['data']):
    this_item = {
        "itemName": taxon_record[0],
        "kvpValues": [
            {"key": "traitName", "value": taxon_record[1]},
            {"key": "traitValue", "value": taxon_record[2]}
        ]
    }
    list_items.append(this_item)
    if count >= limit: break
    
print("list_items", list_items[:30])

list_items [{'itemName': 'Abrodictyum caudatum', 'kvpValues': [{'key': 'traitName', 'value': 'photosynthetic_pathway'}, {'key': 'traitValue', 'value': 'c3'}]}, {'itemName': 'Abrotanella nivigena', 'kvpValues': [{'key': 'traitName', 'value': 'photosynthetic_pathway'}, {'key': 'traitValue', 'value': 'c3'}]}, {'itemName': 'Abrus precatorius', 'kvpValues': [{'key': 'traitName', 'value': 'photosynthetic_pathway'}, {'key': 'traitValue', 'value': 'c3'}]}, {'itemName': 'Abutilon', 'kvpValues': [{'key': 'traitName', 'value': 'photosynthetic_pathway'}, {'key': 'traitValue', 'value': 'c3'}]}, {'itemName': 'Abutilon calliphyllum', 'kvpValues': [{'key': 'traitName', 'value': 'photosynthetic_pathway'}, {'key': 'traitValue', 'value': 'c3'}]}, {'itemName': 'Abutilon fraseri', 'kvpValues': [{'key': 'traitName', 'value': 'photosynthetic_pathway'}, {'key': 'traitValue', 'value': 'c3'}]}, {'itemName': 'Abutilon fraseri subsp. diplotrichum', 'kvpValues': [{'key': 'traitName', 'value': 'photosynthetic_pathw

### Push data to lists.ala.org.au (or lists-test.ala.org.au).

See API docs: https://docs.test.ala.org.au/openapi/index.html?urls.primaryName=specieslist#/Lists/Add%20or%20replace%20a%20species%20list

POST body data example:

CURL example: `curl "api_endpoint_here" -H "Authorization: Bearer {access_token}"`

Note that the `decouple` module can either read a `.env` file at the repository root or via environment variables (e.g. via Airflow)

POST body example:

```json
{
  "listName": "list1",
  "listType": "TEST",
  "listItems": [
    {
      "itemName": "item1",
      "kvpValues": [
        {
          "key": "key1",
          "value": "value1"
        },
        {
          "key": "key2",
          "value": "value2"
        }
      ]
    }
  ]
}
```

In [63]:
import requests
import json
import sys, os
sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'code'))
from access_token import create_token
import decouple

config = decouple.AutoConfig(' ') # this is a hack for Jupyter due to running in a sandbox


# ACCESS_TOKEN = config('ACCESS_TOKEN')
access_token = create_token()
ALA_USER_ID = config('ALA_USER_ID', default=13)
# REFRESH_TOKEN = config('REFRESH_TOKEN')
# print("Checking tokens:",access_token)
# print("Checking userID:",ALA_USER_ID)

ALA_API_BASE_URL = config('ALA_API_BASE_URL', default="https://api.ala.org.au")
# alaUserId = config('ALA_USER_ID') # userID for owner of the list or anyone with admin role
dataResourceId = trait_dr_id # trait_drs[trait_name] 
lists_url = f"{ALA_API_BASE_URL}/specieslist/ws/speciesListPost/{dataResourceId}"
headers = {"Content-Type":"application/json", "Authorization": f"Bearer {access_token}", "X-ALA-userId": f"{ALA_USER_ID}"}

payload = {
    "listName": f"AusTraits {trait_name} trait list",
    "listItems": list_items,
}
json_body = json.dumps(payload)
print("url", lists_url)
# print("body", json_body)
r = requests.post(lists_url, data=json_body, headers=headers)
# print("connection data",lists_url,headers)

if r.ok:
    # data = r.json()
    print("POST success:", r.text, "| status code:", r.status_code)
else:
    print("Unable to POST data. ", r.status_code, r.text)
# print("actual request headers", r.request.headers)

url https://api.test.ala.org.au/specieslist/ws/speciesListPost/dr18707
Unable to POST data.  504 {"message": "Endpoint request timed out"}


In [64]:
print("number of taxa:", len(list_items))

number of taxa: 8547
